In [1]:
!pip install diffusers transformers accelerate

In [4]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe=DiffusionPipeline.from_pretrained("ali-vilab/text-to-video-ms-1.7b",torch_dtype=torch.float16,variant="fp16")
pipe.scheduler=DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()



model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/2.82G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after Diffusers version 0.33.1. 


In [6]:
prompt = "spiderman is surfing"
video_frames=pipe(prompt,num_inference_steps=25,).frames

# The output from `pipe().frames` is typically a torch.Tensor of shape
# `(batch_size, num_frames, num_channels, height, width)`.
# When assigned, it often becomes a numpy array of this shape (e.g., (1, 25, 3, 256, 256) for a single prompt).
# The `export_to_video` function in diffusers expects a 4D array or a list of images.
# It does not handle a 5D array (batch dimension).
# We need to remove the batch dimension if it exists.
if isinstance(video_frames, torch.Tensor):
    video_frames = video_frames.cpu().numpy()

if video_frames.ndim == 5 and video_frames.shape[0] == 1:
    video_frames = video_frames.squeeze(0) # Reduce from (1, N, C, H, W) to (N, C, H, W)

# Now, `video_frames` is (num_frames, channels, height, width) which `export_to_video` can handle.
video_path = export_to_video(video_frames)
video_name= video_path.replace('/tmp','')
print ('Name:',video_name)
torch.cuda.empty_cache()

  0%|          | 0/25 [00:00<?, ?it/s]

Name: wq59_yxb.mp4
